# Rekurencyjne Sieci Neuronowe (RNN)

* 2025/2026, A. Kania

Zagadnienia na dziś:
- Rekurencyjne sieci neuronowe
- LSTM

## Dane sekwencyjne

Modele, którymi zajmowaliśmy się wcześniej zakładały konkretny kształt danych. Dla przykładu klasyczna sieć neuronowa fully-connected dla MNISTa zakładała, że na wejściu dostanie wektory rozmiaru 784 - dla wektorów o innej wymiarowości i innych obiektów model zwyczajnie nie będzie działać.

Takie założenie bywa szczególnie niewygodne przy pracy z niektórymi typami danych, takimi jak:
* językiem naturalny (słowa czy zdania mają zadanej z góry liczby znaków)
* szeregi czasowe (dane giełdowe ciągną się właściwie w nieskończoność) 
* dźwięk (nagrania mogą być krótsze lub dłuższe).

Do rozwiązania tego problemu służą rekuencyjne sieci neuronowe (*recurrent neural networks, RNNs*), które zapamiętują swój stan z poprzedniej iteracji.

In [21]:
data_dir = 'names'

In [82]:
import os
from typing import Tuple, Optional, List
import string
import unicodedata


all_letters = string.ascii_letters
n_letters = len(all_letters)
char_to_idx = {c: i for i, c in enumerate(all_letters)}




category_lines = {}
all_categories = []

data = []
targets = [] 
label_to_idx = {}

def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]

def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)



for label, file_name in enumerate(os.listdir(data_dir)):
    
    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += names
    targets += len(names) * [label]

In [76]:
data[0]

'Khoury'

In [77]:
targets[0]

0

In [50]:
X, y = data, targets

<h3> RNN

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import numpy as np
from sklearn.model_selection import train_test_split

In [51]:
def text_to_onehot(text):
    """Zamienia tekst na sekwencję wektorów one-hot."""
    seq = torch.zeros(len(text), n_letters)
    for i, ch in enumerate(text):
        seq[i, char_to_idx[ch]] = 1.0
    return seq

In [52]:
text_to_onehot("ala")

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [53]:
class ListDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return text_to_onehot(self.data[idx]), torch.tensor(self.targets[idx], dtype=torch.long)

#do obslugi sekwencji o roznej dlugosci
def collate_fn(batch):
    # batch = [(seq1, label1), (seq2, label2), ...]
    sequences, labels = zip(*batch)
    lengths = torch.tensor([len(seq) for seq in sequences])
    padded = pad_sequence(sequences, batch_first=True)  # wymiar: (batch, max_len, vocab_size)
    labels = torch.stack(labels)
    return padded, labels, lengths

In [54]:
indices = np.arange(len(X))
train_ind, test_ind = train_test_split(indices, test_size=0.3, random_state=42, stratify=y)

train_targets = [y[i] for i in train_ind]
test_targets = [y[i] for i in test_ind]

In [55]:
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(train_targets) / counts
weight = [weight_per_class[c] for c in train_targets]
sampler = WeightedRandomSampler(weights=weight, num_samples=len(weight))

train_dataset = ListDataset([X[i] for i in train_ind], train_targets)
test_dataset = ListDataset([X[i] for i in test_ind], test_targets)

train_loader = DataLoader(train_dataset, batch_size=4, sampler=sampler, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [56]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, lengths):
        # Pack sequences
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, hidden = self.rnn(packed)
        # hidden: (1, batch, hidden_size)
        out = self.fc(hidden.squeeze(0))
        return out

In [61]:
num_classes = len(set(y))
model = RNNClassifier(input_size=n_letters, hidden_size=32, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [62]:
for epoch in range(5):
    model.train()
    total_loss = 0
    for X_batch, y_batch, lengths in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch, lengths)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


Epoch 1, Loss: 1.7955
Epoch 2, Loss: 1.5811
Epoch 3, Loss: 1.5504
Epoch 4, Loss: 1.6250
Epoch 5, Loss: 1.6921


In [63]:
def evaluate(model, loader, topk=(1,3)):
    model.eval()
    correct = {k: 0 for k in topk}
    total = 0

    with torch.no_grad():
        for X_batch, y_batch, lengths in loader:
            outputs = model(X_batch, lengths)
            total += y_batch.size(0)
            for k in topk:
                _, pred = outputs.topk(k, dim=1)
                correct[k] += (pred == y_batch.view(-1, 1)).any(dim=1).sum().item()

    for k in topk:
        acc = 100 * correct[k] / total
        print(f"Top-{k} accuracy: {acc:.2f}%")

print("\n=== Test results ===")
evaluate(model, test_loader)


=== Test results ===
Top-1 accuracy: 42.32%
Top-3 accuracy: 69.68%


<h3> LSTM</h3>

In [65]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths):
        # x: (batch, seq_len, input_size)
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, (hidden, cell) = self.lstm(packed)
        # hidden: (num_layers, batch, hidden_size)
        out = self.dropout(hidden[-1])  # ostatnia warstwa, ostatni stan
        out = self.fc(out)
        return out

# === 7. Inicjalizacja ===
num_classes = len(set(y))
model = LSTMClassifier(input_size=n_letters, hidden_size=64, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# === 8. Trening ===
for epoch in range(5):
    model.train()
    total_loss = 0
    for X_batch, y_batch, lengths in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch, lengths)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1:02d} | Loss: {total_loss / len(train_loader):.4f}")

# === 9. Ewaluacja ===
def evaluate(model, loader, topk=(1,3)):
    model.eval()
    correct = {k: 0 for k in topk}
    total = 0

    with torch.no_grad():
        for X_batch, y_batch, lengths in loader:
            outputs = model(X_batch, lengths)
            total += y_batch.size(0)
            for k in topk:
                _, pred = outputs.topk(k, dim=1)
                correct[k] += (pred == y_batch.view(-1, 1)).any(dim=1).sum().item()

    for k in topk:
        acc = 100 * correct[k] / total
        print(f"Top-{k} accuracy: {acc:.2f}%")

print("\n=== Test results ===")
evaluate(model, test_loader)

Epoch 01 | Loss: 1.5089
Epoch 02 | Loss: 0.8308
Epoch 03 | Loss: 0.6382
Epoch 04 | Loss: 0.5455
Epoch 05 | Loss: 0.5003


KeyboardInterrupt: 

In [91]:
import pandas as pd

df = pd.read_csv("all_chem_df.csv", sep = ",")

from collections import Counter
print(Counter(df["tags"]).most_common(4))

df2 = df[df['tags'].isin(['antiinfective', 'antineoplastic', 'cns','cardio'])]
print(df2.head())

wszystkie = list(df2["smiles"])

zlaczone = "".join(list(df2["smiles"]))

[('antiinfective', 2412), ('antineoplastic', 1175), ('cns', 1149), ('cardio', 797)]
  image_name            tags  \
1     pics/1   antiinfective   
2     pics/2   antiinfective   
3     pics/3  antineoplastic   
5     pics/5             cns   
6     pics/6             cns   

                                              smiles                Col3  
1  CCC[C@@]1(CCc2ccccc2)CC(O)=C([C@H](CC)c2cccc(N...   ['antiinfective']  
2  CCCCC(C)C(=O)OC1C(C)C(CC)OC2(CC3CC(C/C=C(\C)CC...   ['antiinfective']  
3  COc1cc2c(c(OC)c1OC)-c1c(cc3c(c1OC)OCO3)C[C@H](...  ['antineoplastic']  
5  CC(=O)OCC(=O)C1CCC2C3CCC4CC(O)CCC4(C)C3C(=O)CC12C             ['cns']  
6                         CC(=O)Nc1nnc(S(N)(=O)=O)s1             ['cns']  


In [92]:
X = wszystkie
print(X[0])

CCC[C@@]1(CCc2ccccc2)CC(O)=C([C@H](CC)c2cccc(NS(=O)(=O)c3ccc(C(F)(F)F)cn3)c2)C(=O)O1


In [93]:
kategorie = {'antiinfective':0, 'antineoplastic':1, 'cns':2,'cardio':3}
y = [kategorie[elem] for elem in df2["tags"]]
print(y[:3])

[0, 0, 1]


In [104]:
all_letters = "".join(list(set(zlaczone))) #wszystkie symbole w smiles
n_letters = len(all_letters)
char_to_idx = {c: i for i, c in enumerate(all_letters)}

In [105]:
indices = np.arange(len(X))
train_ind, test_ind = train_test_split(indices, test_size=0.3, random_state=42, stratify=y)

train_targets = [y[i] for i in train_ind]
test_targets = [y[i] for i in test_ind]

uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(train_targets) / counts
weight = [weight_per_class[c] for c in train_targets]
sampler = WeightedRandomSampler(weights=weight, num_samples=len(weight))

train_dataset = ListDataset([X[i] for i in train_ind], train_targets)
test_dataset = ListDataset([X[i] for i in test_ind], test_targets)

train_loader = DataLoader(train_dataset, batch_size=4, sampler=sampler, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [106]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths):
        # x: (batch, seq_len, input_size)
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, (hidden, cell) = self.lstm(packed)
        # hidden: (num_layers, batch, hidden_size)
        out = self.dropout(hidden[-1])  # ostatnia warstwa, ostatni stan
        out = self.fc(out)
        return out

# === 7. Inicjalizacja ===
num_classes = len(set(y))
model = LSTMClassifier(input_size=n_letters, hidden_size=64, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# === 8. Trening ===
for epoch in range(5):
    model.train()
    total_loss = 0
    for X_batch, y_batch, lengths in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch, lengths)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1:02d} | Loss: {total_loss / len(train_loader):.4f}")

# === 9. Ewaluacja ===
def evaluate(model, loader, topk=(1,2)):
    model.eval()
    correct = {k: 0 for k in topk}
    total = 0

    with torch.no_grad():
        for X_batch, y_batch, lengths in loader:
            outputs = model(X_batch, lengths)
            total += y_batch.size(0)
            for k in topk:
                _, pred = outputs.topk(k, dim=1)
                correct[k] += (pred == y_batch.view(-1, 1)).any(dim=1).sum().item()

    for k in topk:
        acc = 100 * correct[k] / total
        print(f"Top-{k} accuracy: {acc:.2f}%")

print("\n=== Test results ===")
evaluate(model, test_loader)

Epoch 01 | Loss: 1.3471
Epoch 02 | Loss: 1.2586
Epoch 03 | Loss: 1.1129
Epoch 04 | Loss: 1.1046
Epoch 05 | Loss: 0.9517

=== Test results ===
Top-1 accuracy: 60.54%
Top-2 accuracy: 80.30%


<h4> Zadanie z przetwarzaniem sekwencji białkowych, pobawic sie roznymi tokenami i reprezentacjami (embedingmai):
- np 1 aminokwas - 1 slowo - [hydrofoboossc, pI, gravy] <- embedding
- sekwencjei nukleoptydowe - np 3 kolejne slowa, arbitralnie, embeddingi 64D

powiedziec, ze jak tekst to tokenem moze byc 1 znak, 1 slowo a aczasem cos pomiędzy i zaprosic na kurs z NLP, powiedziec ze tam jest tylko o przetwarzaniu tekstu

Pokazac: https://platform.openai.com/tokenizer I like bioinformatics

Zadanie: jakis gotowy model Transformer do dotrenowania

https://aisingapore.org/animated-rnn-lstm-gru/

Obrazek rnn i gify rnn pokazac

<h4> Fine-tuning

In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

ModuleNotFoundError: No module named 'transformers'

https://chatgpt.com/c/6914d5ba-217c-832c-bcad-69048cfe4499

https://chatgpt.com/c/69199e9e-2b1c-8328-b7e5-3ff5dc5e0201

<h4> Modele z huggin face

In [3]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [16]:
texts = ["a dog", "a happy dog", "a cat", "a happy cat", "grass"]

image = Image.open("cat.jpg")

inputs = processor(
    text=texts,
    images=image,
    return_tensors="pt",
    padding=True,
    truncation=True
)

outputs = model(**inputs)
probs = outputs.logits_per_image.softmax(dim=1)
print(probs)

tensor([[0.0011, 0.0021, 0.1344, 0.8516, 0.0108]], grad_fn=<SoftmaxBackward0>)


Tekst -> Obrazek

In [18]:
from diffusers import StableDiffusionPipeline
import torch

In [19]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/ss")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-6919a867-20683b056c92be164cee9e66;5aeda580-b9c4-4836-8ec2-777d46848337)

Repository Not Found for url: https://huggingface.co/api/models/CompVis/ss.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password..
Will try to load from local cache.


OSError: Cannot load model CompVis/ss: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

In [1]:
image = pipe("very happy dog").images[0] #ok 15/20 min sie generuje
image.save("happy_dog.png")

NameError: name 'pipe' is not defined